# Iterating through the list of authors

In [1]:
import pickle
import re
import pandas as pd
from bs4 import BeautifulSoup
from parsing_page import parsing_author_page, parsing_article_page
import time
import requests
from os.path import exists
from os import makedirs


In [2]:
from data_types import Author, AuthorsDB
from data_types import Publication, PublicationsDB
from data_types import Abstract, AbstractsDB
audb = AuthorsDB()
# audb.load()
pubdb = PublicationsDB()
# pubdb.load()
absdb = AbstractsDB()
# absdb.load()

In [3]:
filename = "../data/mathnet_iam_authors_dict.pkl"
with open(filename,'rb') as inp:
    authors_dict = pickle.load(inp)

# Main functions for 

In [4]:
def need_to_scrap(author, authors_db:AuthorsDB):
    interesting_list = ["Бондаренко", "Поляков", "Якобовский", "Четверушкин"]
    # interesting_list = ["Бондаренко", "Якобовский"]
    # interesting_list = ["Бондаренко"]
    for person in interesting_list:
        if person in author['name']:
            # we can demand some other conditions
            # if not authors_db.check_key(author["mn_id"]):
            if authors_db.check_key(author["mn_id"]):
                print(f"We have him {author} in authors_db")
                return True
            else:
                # print(f"We have him {author} in authors_db")
                print(f"We don't have him {author} in authors_db")
                return True
    return False 

In [5]:
def save_html(data, mnid, name, status='write'):
    newpath = r'../data/'+mnid+'/'
    if not exists(newpath):
        makedirs(newpath)
    filename = "../data/"+mnid+"/"+name+".pkl"
    if status == 'write':        
        with open(filename,'wb') as outp:
            pickle.dump(data, outp, pickle.HIGHEST_PROTOCOL)

In [6]:
def get_author_info(mnid, status="not_saving", web='offline'):
    print("../data/"+mnid+"/mn_author_"+mnid+"_page.pkl")
    print(exists("../data/"+mnid+"/mn_author_"+mnid+"_page.pkl"))
    if exists("../data/"+mnid+"/mn_author_"+mnid+"_page.pkl"):
        print("We read from file")
        filename = "../data/"+mnid+"/mn_author_"+mnid+"_page.pkl"
        with open(filename,'rb') as inp:
            author_page = pickle.load(inp)
    else:
        print(f"we need to request page for author mnid = {mnid}")
        if web=='offline':
            return None
        page_link = f"http://www.mathnet.ru/php/person.phtml?option_lang=rus&personid={mnid}"
        
        # Chancge 'User-Agent' HEADERRRRR after some time of use !!!!
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        # page_link ="http://mi.mathnet.ru/+ {}"
        time.sleep(0.7)
        response = requests.get(page_link, headers=headers, timeout=None)
        if response.status_code == 200:
            author_page = response.content
        if status=="saving":
            save_html(author_page, mnid, "mn_author_"+mnid+"_page",status='write')
        author_page
                    
    soup = BeautifulSoup(author_page, 'html.parser')
    # TODO:
    # Need to check if this is real page and not for bot response
    return parsing_author_page(soup) 

In [7]:
def get_pub_info(mn_id, mnlink, status="not_saving",web='offline'):
    print("../data/"+mn_id+"/"+mnlink+".pkl")
    print(exists("../data/"+mn_id+"/"+mnlink+".pkl"))
    if exists("../data/"+mn_id+"/"+mnlink+".pkl"):
        print("We read from file")
        filename = "../data/"+mn_id+"/"+mnlink+".pkl"
        with open(filename,'rb') as inp:
            pub_page = pickle.load(inp)
    else:
        print("we don't find that file")
        if web == 'offline':
            return None   
        print(f"we need to request page for pub_page mnlink = {mnlink}")
        page_link = "http://mi.mathnet.ru/" + mnlink
        print(page_link)
        # Chancge 'User-Agent' HEADERRRRR after some time of use !!!!
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        # 
        time.sleep(0.5)
        response = requests.get(page_link, headers=headers, timeout=None)        
        print(response.status_code)        
        if response.status_code == 200:
            pub_page = response.content            
            print("we save file: " + mnlink)
            if status=="saving":
                save_html(pub_page, mn_id, mnlink,status='write')
        else:
            print(response.status_code)
            print(type(response.status_code))
            print("response.status_code != 200:")
            return "Stop"
        # pub_page        
    soup = BeautifulSoup(pub_page, 'html.parser')
    # if status != "working":
    #     return parsing_article_page(soup)
    
    return parsing_article_page(soup)

In [8]:
audb.show()

printing AuthorsDB:


In [9]:
save_html_status = "saving"
status = "testing"
# safe
cur_web = 'offline'
#saveingDB = 'easy'
# -------------
# working
cur_web = 'online'
saveingDB = 'write'
for indx, item in authors_dict.items(): 
    request_problems = False
    if need_to_scrap(item, audb):                
        new_author = None
        new_author = Author(mn_id=item["mn_id"])
        print(item)
        # get through pubs list on author page
        author_pubs = None
        author_pubs = get_author_info(item["mn_id"], status=save_html_status,web=cur_web)
        if author_pubs is None:
            continue
        for pubs_indx, pubs_item in author_pubs.items():
            if 'mn_link' not in pubs_item:
                continue                
            print(pubs_indx, pubs_item)            
            if pubs_item["mn_link"] in pubdb.db.keys():
                # статья уже есть надо просто добавить к автору
                new_author.read_from_pubdb(pubs_item["mn_link"], audb)                
                continue
            
            if pubs_item["mn_link"] is not None:
                pub_page_info = None                                
                pub_page_info = get_pub_info(item["mn_id"],pubs_item["mn_link"],status=save_html_status,web=cur_web)
                # if len(pub_page_info)==1:
                # # if pub_page_info=="Stop":
                #     audb.save(f"we have len(pub_page_info)==1 \nsaving after author={item['mn_id']}, link={pubs_item['mn_link']}")
                #     pubdb.save()
                #     absdb.save()
                #     # request_problems = True
                #     continue
                
                    
                new_pub = Publication(pubs_item["mn_link"])
                new_abs = Abstract(pubs_item["mn_link"])
                
                if pub_page_info is not None:                    
                    new_author.update_author_info(pubs_item, pub_page_info)
                    new_pub.update_publication_info(pubs_item, pub_page_info)
                    new_abs.update_abstract_info(pubs_item, pub_page_info)                    
                    pubdb.update_data(new_pub.convert2dict())                            
                    absdb.update_data(new_abs.convert2dict())
                    for page_indx, page_item  in pub_page_info.items():
                        print(page_indx)
                        print(page_item)                              
        if request_problems == True:
            break
        print(new_author.convert2dict())
        audb.update_data(new_author.convert2dict())
        if status  == "testing":
            audb.show()
            pubdb.show()
            absdb.show()
        audb.save(status = saveingDB,text = f"we saving after {item['mn_id']} author")
        pubdb.save(status = saveingDB)
        absdb.save(status = saveingDB)

We don't have him {'name': 'Бондаренко Алексей Алексеевич', 'mn_id': '113970'} in authors_db
{'name': 'Бондаренко Алексей Алексеевич', 'mn_id': '113970'}
../data/113970/mn_author_113970_page.pkl
True
We read from file
1 {'name': 'А. А. Бондаренко, П. А. Ляхов, М. В. Якобовский, “Координированное сохранение с журналированием передаваемых данных и асинхронное восстановление в случае отказа”, Вестн. ЮУрГУ. Сер. Выч. матем. информ., 8:2 (2019), 76–91', 'mn_link': 'vyurv213', 'elib_id': '38073495'}
../data/113970/vyurv213.pkl
True
We read from file
{'abstract': 'Увеличивающийся рост числа компонент суперкомпьютеров приводит специалистов в области HPC к неблагоприятным оценкам для будущих суперкомпьютеров: диапазон среднего времени между отказами будет составлять от 1 часа до 9 часов. Данная оценка ставит под вопрос возможность проведения длительных расчетов на суперкомпьютерах. В работе предлагается метод восстановления после отказов, не требующий возврата большинства процессов к последней 

In [10]:
audb.show()

printing AuthorsDB:
ind = 113970
item = {'vyurv213': {'year': '2019', 'nrf': 2}, 'vyurv1': {'year': '2015', 'nrf': 2}, 'vyurv46': {'year': '2014', 'nrf': 4}}
ind = 25339
item = {'ipmp2988': {'year': '2021', 'nrf': 6}, 'ipmp2939': {'year': '2021', 'nrf': 6}, 'mm4253': {'year': '2021', 'nrf': 2}, 'danma122': {'year': '2020', 'nrf': 9}, 'vmp992': {'year': '2020', 'nrf': 2}, 'ipmp2783': {'year': '2019', 'nrf': 6}, 'ipmp2734': {'year': '2019', 'nrf': 6}, 'mm4137': {'year': '2019', 'nrf': 2}, 'mm4110': {'year': '2019', 'nrf': 2}, 'mm4102': {'year': '2019', 'nrf': 2}, 'ipmp2604': {'year': '2018', 'nrf': 6}, 'vmp897': {'year': '2018', 'nrf': 4}, 'ipmp2285': {'year': '2017', 'nrf': 6}, 'mm3753': {'year': '2016', 'nrf': 3}, 'tisp31': {'year': '2016', 'nrf': 3}, 'vmp823': {'year': '2016', 'nrf': 4}, 'crm221': {'year': '2015', 'nrf': 1}, 'mm3627': {'year': '2015', 'nrf': 3}, 'mm3624': {'year': '2015', 'nrf': 3}, 'mm3578': {'year': '2015', 'nrf': 2}, 'tisp206': {'year': '2015', 'nrf': 3}, 'vmp525':

In [11]:
pubdb.show()

printing PublicationsDB:
ind = vyurv213
item = {'author_id': ['113970', '148811', '22428'], 'doi': '<a class="SLink" href="https://doi.org/10.14529/cmse190205" target="_blank" title="DOI: https://doi.org/10.14529/cmse190205">https://doi.org/10.14529/cmse190205', 'udk': '004.052.3', 'send': ' 20.11.2018', 'type': 'Статья', 'reference': '', 'by': 'А.~А.~Бондаренко, П.~А.~Ляхов, М.~В.~Якобовский', 'paper': 'Координированное сохранение с журналированием передаваемых данных и асинхронное восстановление в случае отказа', 'jour': 'Вестн. ЮУрГУ. Сер. Выч. матем. информ.', 'yr': '2019', 'vol': '8', 'issue': '2', 'pages': '76--91'}
ind = vyurv1
item = {'author_id': ['113970', '22428'], 'doi': '<a class="SLink" href="https://doi.org/10.14529/cmse150301" target="_blank" title="DOI: https://doi.org/10.14529/cmse150301">https://doi.org/10.14529/cmse150301', 'udk': '004.052.3', 'send': ' 13.04.2015', 'type': 'Статья', 'reference': '', 'by': 'А.~А.~Бондаренко, М.~В.~Якобовский', 'paper': 'Моделировани

In [12]:
absdb.show()

printing AbstractsDB:
ind =
vyurv213
row =
abstract    Увеличивающийся рост числа компонент суперкомп...
keywords    расширение ULFM, контрольные точки, координиро...
Name: vyurv213, dtype: object
ind =
vyurv1
row =
abstract    Рассматривается проблема выполнения длительных...
keywords    параллельные вычисления, отказоустойчивость, к...
Name: vyurv1, dtype: object
ind =
vyurv46
row =
abstract    Рассматриваются вопросы, связанные с проведени...
keywords    параллельные вычисления, отказоустойчивость, к...
Name: vyurv46, dtype: object
ind =
ipmp2988
row =
abstract    В данной работе рассмотрена модель для сопряже...
keywords    модель Бринкмана, уравнения Навье-Стокса, своб...
Name: ipmp2988, dtype: object
ind =
ipmp2939
row =
abstract    Данное исследование связано с разработкой мате...
keywords    математическое моделирование, решетка графена,...
Name: ipmp2939, dtype: object
ind =
mm4253
row =
abstract    Рассматривается проблема моделирования свойств...
keywords    свойства техниче